In [1]:
import pandas as pd

csv_file = 'tabular_findings.csv'
df = pd.read_csv(csv_file, dtype=str)
df = df.drop_duplicates()

In [6]:
df

,Unnamed: 0,Aorta Sinuses of Valsalva Quality,Mitral Valve Leaflets,Aortic Valve AoV Mean PG,Tricuspid Valve Prolapse,Mitral Valve Grade,Tricuspid Valve Rheumatic Valve,Left Ventricle Max Wall Thickness,Pulmonary Valve RVOT Vmax,Pericardium Fat pad,...,Aorta Descending Aorta Quality,Tricuspid Valve Mobility,Inferior Vena Cava Dilated,Pulmonary Valve Dilated,Aortic Valve AoV Peak PG,Tricuspid Valve Mass,Pulmonary Valve Leaflet thickening,Mitral Valve Prolapse,Mitral Valve Rheumatic Valve,Mitral Valve SAM structure
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,Normal,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,Normal,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,Normal,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69669,69669,NaN,NaN,#,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,#,NaN,Normal,NaN,NaN,NaN
69670,69670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,Normal,NaN,NaN,NaN
69671,69671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69672,69672,NaN,NaN,#,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Normal,NaN,NaN,NaN,NaN,Normal,NaN,NaN,NaN


In [7]:
import pandas as pd

csv_file = 'reports_df.csv'
df = pd.read_csv(csv_file)
df = df.drop_duplicates()

df['findings_txt'] = df['findings_txt'].apply(lambda x: bytes(x, "utf-8").decode("unicode_escape"))

df.shape[0]
df = df.head(1)

In [77]:
featuresDf = pd.read_csv('LLM-Categorization-v2.csv')
featuresDf = featuresDf.drop([featuresDf.columns[4], featuresDf.columns[6], featuresDf.columns[7], featuresDf.columns[8], featuresDf.columns[9]], axis=1)
for column in featuresDf.columns:
    featuresDf[column].fillna(method='ffill', inplace=True)
print(featuresDf)

uniqueFindingsDict = {}
sections = set()
for index, row in featuresDf.iterrows():
    uniqueFindingsDict[row['Section'] + " " + row['Findings/Unique Sentences']] = row['Section'] + " " + row['Features']
    sections.add(row['Section'])
print(sections)

            Section         Subsection           Features       Classes  \
0    Left Ventricle  Systolic Function  Systolic Function  Hyperdynamic   
1    Left Ventricle  Systolic Function  Systolic Function        Normal   
2    Left Ventricle  Systolic Function  Systolic Function        Normal   
3    Left Ventricle  Systolic Function  Systolic Function          Mild   
4    Left Ventricle  Systolic Function  Systolic Function      Moderate   
..              ...                ...                ...           ...   
854      Additional         Ventilated         Ventilated       Boolean   
855      Additional         Ventilated         Ventilated       Boolean   
856      Additional         Ventilated         Ventilated       Boolean   
857      Additional         Ventilated         Ventilated       Boolean   
858      Additional         Ventilated         Ventilated       Boolean   

                       Findings/Unique Sentences  
0                 Hyperdynamic systolic function

In [78]:
import pandas as pd

# Initialize a DataFrame with the same number of rows as df and columns as the values of uniqueFindingsDict.
tabularFindings = pd.DataFrame(index=df.index, columns=list(set(uniqueFindingsDict.values())))

for index, row in df.iterrows():
    findingsSplitBySection = row['findings_txt'].splitlines()
    for section in sections:
        findingsSection = [s for s in findingsSplitBySection if s.startswith(section)]
        if len(findingsSection)>0:
            sectionFeatures = featuresDf[featuresDf['Section'] == section]
            for sectionIndex, sectionRow in sectionFeatures.iterrows():
                if sectionRow['Findings/Unique Sentences'] in findingsSection[0]:
                    tabularFindings.loc[index, sectionRow['Section'] + " " + sectionRow['Features']] = sectionRow['Classes']
print(tabularFindings)

      Aorta Sinuses of Valsalva Quality Mitral Valve Leaflets  \
0                                   NaN                   NaN   
1                                   NaN                   NaN   
2                                   NaN                   NaN   
3                                   NaN                   NaN   
4                                   NaN                   NaN   
...                                 ...                   ...   
69669                               NaN                   NaN   
69670                               NaN                   NaN   
69671                               NaN                   NaN   
69672                               NaN                   NaN   
69673                               NaN                   NaN   

      Aortic Valve AoV Mean PG Tricuspid Valve Prolapse Mitral Valve Grade  \
0                          NaN                      NaN                NaN   
1                          NaN                      NaN        

In [2]:
tabularFindings.to_csv('tabular_findings.csv')

NameError: name 'tabularFindings' is not defined

In [5]:
import pandas as pd

# Initialize a DataFrame with the same number of rows as df and columns as features_lv.
results = pd.DataFrame(index=df.index, columns=features_lv)

for index, row in df.iterrows():
    lv_text = re.search('Left Ventricle:(.*?)(?=\n[A-Z])', row['findings_txt'], re.DOTALL)
    if lv_text is not None:
        findings_text = lv_text.group(1)
        sentences = findings_text.split('.')  # Splitting by '.'

        for sentence in sentences:
            sentence = sentence.strip()  # remove leading/trailing whitespaces
            for feature in features_lv:
                if feature.lower() in sentence.lower():
                    results.loc[index, feature] = sentence
    else:
        print(f"Row {index}: No findings for Left Ventricle")

print(results)


Row 10145: No findings for Left Ventricle
Row 10183: No findings for Left Ventricle
Row 10296: No findings for Left Ventricle
Row 10391: No findings for Left Ventricle
Row 11155: No findings for Left Ventricle
Row 11473: No findings for Left Ventricle
Row 11740: No findings for Left Ventricle
Row 11748: No findings for Left Ventricle
Row 11769: No findings for Left Ventricle
Row 11840: No findings for Left Ventricle
Row 11936: No findings for Left Ventricle
Row 12145: No findings for Left Ventricle
Row 12931: No findings for Left Ventricle
Row 13325: No findings for Left Ventricle
Row 13399: No findings for Left Ventricle
Row 13872: No findings for Left Ventricle
Row 14293: No findings for Left Ventricle
Row 14842: No findings for Left Ventricle
Row 15329: No findings for Left Ventricle
Row 15646: No findings for Left Ventricle
Row 16808: No findings for Left Ventricle
Row 17107: No findings for Left Ventricle
Row 17367: No findings for Left Ventricle
Row 17641: No findings for Left Ve